## Weekly Twitter, Facebook and Instagram Organic Data

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from time import strptime

### Twitter
Read in data export from twitter platform, wrangle, and export cleaned csv

In [2]:
# 12/6 - 12/12
twitter = pd.read_csv('twitter_weekly_pull.csv')
print(twitter.info())
twitter.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 40 columns):
Tweet id                        116 non-null float64
Tweet permalink                 116 non-null object
Tweet text                      116 non-null object
time                            116 non-null object
impressions                     116 non-null int64
engagements                     116 non-null int64
engagement rate                 116 non-null float64
retweets                        116 non-null int64
replies                         116 non-null int64
likes                           116 non-null int64
user profile clicks             116 non-null int64
url clicks                      116 non-null int64
hashtag clicks                  116 non-null int64
detail expands                  116 non-null int64
permalink clicks                116 non-null int64
app opens                       116 non-null int64
app installs                    116 non-null int64
follows              

,Tweet id,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,...,promoted hashtag clicks,promoted detail expands,promoted permalink clicks,promoted app opens,promoted app installs,promoted follows,promoted email tweet,promoted dial phone,promoted media views,promoted media engagements
0,1.072540e+18,https://twitter.com/KUWTK/status/1072540945810...,.@KrisJenner called in the big guns to cheer u...,2018-12-11 17:17 +0000,15831,1115,0.070431,27,2,255,...,-,-,-,-,-,-,-,-,-,-
1,1.072170e+18,https://twitter.com/KUWTK/status/1072174102960...,The fam has been #blessed this past year 🙏 #KU...,2018-12-10 17:00 +0000,126587,12874,0.101701,129,25,1110,...,-,-,-,-,-,-,-,-,-,-
2,1.072010e+18,https://twitter.com/KUWTK/status/1072008006353...,"It's been real, West Coast ✌️ Try not to miss ...",2018-12-10 06:00 +0000,14432,309,0.021411,12,17,168,...,-,-,-,-,-,-,-,-,-,-
3,1.072010e+18,https://twitter.com/KUWTK/status/1072007502898...,RT if this season of #KUWTK has been absolutel...,2018-12-10 05:58 +0000,12383,499,0.040297,27,5,114,...,-,-,-,-,-,-,-,-,-,-
4,1.072010e+18,https://twitter.com/KUWTK/status/1072005489897...,The moment Alice first met @KimKardashian 🤗❤️ ...,2018-12-10 05:50 +0000,14097,754,0.053487,22,9,116,...,-,-,-,-,-,-,-,-,-,-


In [3]:
# filter out unneeded columns
twitter = twitter.filter(['Tweet permalink', 'Tweet text', 'time', 'impressions', 'engagements',
                          'engagement rate', 'retweets', 'replies', 'likes', 'media views', 'media engagements'])
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 11 columns):
Tweet permalink      116 non-null object
Tweet text           116 non-null object
time                 116 non-null object
impressions          116 non-null int64
engagements          116 non-null int64
engagement rate      116 non-null float64
retweets             116 non-null int64
replies              116 non-null int64
likes                116 non-null int64
media views          116 non-null int64
media engagements    116 non-null int64
dtypes: float64(1), int64(7), object(3)
memory usage: 10.0+ KB


In [4]:
#create 'post type' column indicating whether reply or original post
twitter['post type'] = twitter['Tweet text'].map(lambda x: "reply" if x[0] == '@' else "post")

# create 'vis eng' column calculating visible engagements by adding retweets, replies, likes
twitter['vis eng'] = twitter['retweets'] + twitter['replies'] + twitter['likes']

# create 'view Rate' and 'completion rate' columns by dividing media views and engagements by impressions, respectively
twitter['view rate'] = twitter['media views']/twitter['impressions']
twitter['completion rate'] = twitter['media engagements']/twitter['impressions']

# add 'episode' column assigning episode number: first 2 digits = season number, last 2 digits = episode number
twitter['episode'] = int(1516)

# convert 'date' column to datetime
twitter['time'] = pd.to_datetime(twitter['time'])

# create 'in-show' column - show aired on 12/9, so tweets between GMT 12/10 1AM - 6AM are "in-show"
twitter['in-show'] = twitter['time'].map(lambda x: "in-show" if x >= pd.to_datetime(['2018-12-10 01:00:00']) and x <= pd.to_datetime(['2018-12-10 06:00:00']) else "not")

# create empty 'Votes' column
twitter['votes'] = np.nan

print(twitter.info())
twitter.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 18 columns):
Tweet permalink      116 non-null object
Tweet text           116 non-null object
time                 116 non-null datetime64[ns]
impressions          116 non-null int64
engagements          116 non-null int64
engagement rate      116 non-null float64
retweets             116 non-null int64
replies              116 non-null int64
likes                116 non-null int64
media views          116 non-null int64
media engagements    116 non-null int64
post type            116 non-null object
vis eng              116 non-null int64
view rate            116 non-null float64
completion rate      116 non-null float64
episode              116 non-null int64
in-show              116 non-null object
votes                0 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(9), object(4)
memory usage: 16.4+ KB
None


,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,media views,media engagements,post type,vis eng,view rate,completion rate,episode,in-show,votes
0,https://twitter.com/KUWTK/status/1072540945810...,.@KrisJenner called in the big guns to cheer u...,2018-12-11 17:17:00,15831,1115,0.070431,27,2,255,3317,767,post,284,0.209526,0.048449,1516,not,NaN
1,https://twitter.com/KUWTK/status/1072174102960...,The fam has been #blessed this past year 🙏 #KU...,2018-12-10 17:00:00,126587,12874,0.101701,129,25,1110,30321,11024,post,1264,0.239527,0.087086,1516,not,NaN
2,https://twitter.com/KUWTK/status/1072008006353...,"It's been real, West Coast ✌️ Try not to miss ...",2018-12-10 06:00:00,14432,309,0.021411,12,17,168,0,0,post,197,0.000000,0.000000,1516,in-show,NaN
3,https://twitter.com/KUWTK/status/1072007502898...,RT if this season of #KUWTK has been absolutel...,2018-12-10 05:58:00,12383,499,0.040297,27,5,114,2960,293,post,146,0.239037,0.023661,1516,in-show,NaN
4,https://twitter.com/KUWTK/status/1072005489897...,The moment Alice first met @KimKardashian 🤗❤️ ...,2018-12-10 05:50:00,14097,754,0.053487,22,9,116,2543,528,post,147,0.180393,0.037455,1516,in-show,NaN


In [5]:
#standardize column names i.e. make all lowercase
twitter.columns = map(str.lower, twitter.columns)
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 18 columns):
tweet permalink      116 non-null object
tweet text           116 non-null object
time                 116 non-null datetime64[ns]
impressions          116 non-null int64
engagements          116 non-null int64
engagement rate      116 non-null float64
retweets             116 non-null int64
replies              116 non-null int64
likes                116 non-null int64
media views          116 non-null int64
media engagements    116 non-null int64
post type            116 non-null object
vis eng              116 non-null int64
view rate            116 non-null float64
completion rate      116 non-null float64
episode              116 non-null int64
in-show              116 non-null object
votes                0 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(9), object(4)
memory usage: 16.4+ KB


In [6]:
#reorder columns
twitter = twitter[['post type', 'tweet permalink', 'tweet text', 'episode', 'time', 'in-show', 'impressions',
                   'engagements', 'engagement rate', 'retweets', 'replies', 'likes', 'votes', 'vis eng',
                   'media views', 'media engagements', 'view rate', 'completion rate']]
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 18 columns):
post type            116 non-null object
tweet permalink      116 non-null object
tweet text           116 non-null object
episode              116 non-null int64
time                 116 non-null datetime64[ns]
in-show              116 non-null object
impressions          116 non-null int64
engagements          116 non-null int64
engagement rate      116 non-null float64
retweets             116 non-null int64
replies              116 non-null int64
likes                116 non-null int64
votes                0 non-null float64
vis eng              116 non-null int64
media views          116 non-null int64
media engagements    116 non-null int64
view rate            116 non-null float64
completion rate      116 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(9), object(4)
memory usage: 16.4+ KB


In [7]:
# convert new dataframe to csv "twitter_weekly.csv"
twitter.to_csv('twitter_weekly.csv')

### Nielsen
Read in data export from Nielsen UI, wrangle and write to csv.

In [8]:
# read in file for current week's episode
# air time: 12/9 - 9pm ET
nielsen = pd.read_csv('nielsen_weekly_pull.csv')
nielsen.info()
nielsen.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16110 entries, 0 to 16109
Data columns (total 14 columns):
Social Content Ratings    16102 non-null object
Unnamed: 1                16100 non-null object
Unnamed: 2                16091 non-null object
Unnamed: 3                16091 non-null object
Unnamed: 4                16091 non-null object
Unnamed: 5                16087 non-null object
Unnamed: 6                16087 non-null object
Unnamed: 7                16085 non-null object
Unnamed: 8                16085 non-null object
Unnamed: 9                16085 non-null object
Unnamed: 10               16085 non-null object
Unnamed: 11               16085 non-null object
Unnamed: 12               16085 non-null object
Unnamed: 13               14445 non-null object
dtypes: object(14)
memory usage: 1.7+ MB


,Social Content Ratings,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,http://scr.nielsensocial.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Copyright 2018 The Nielsen Company. All right...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Network,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Program,Keeping Up With The Kardashians,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Program Genre Type,Comedy Series; Reality Series; Docs & Special ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Season Number,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Episode,Break Free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Episode Number,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,EPG Program ID,SH009675940000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# toss row indices 0-9 into a new dataframe and isolate episode name
episode_details = nielsen.iloc[:9]
season = episode_details.iat[6,1]
episode_details['episode'] = episode_details.iat[6,1] + episode_details.iat[8,1]
episode_details['name'] = episode_details.iat[7,1]
episode_details.drop(episode_details.index[1:],inplace=True)
episode_details = episode_details.loc[:, ['episode','name']]
episode_details

/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydat

,episode,name
0,1516,Break Free


In [10]:
# bring up row index 17 as header and drop row indices 0-16
nielsen.drop(nielsen.index[:17], inplace=True)
nielsen.columns = nielsen.iloc[0]
nielsen = nielsen[1:]
nielsen = nielsen.reset_index(drop = True)
nielsen.head(15)

17,nan,Unique Authors,Original Content,Unique Engagers,Total Engagement,Uniques,Interactions,nan,nan,nan,nan,nan,nan,nan
0,Twitter,862,1640,73856,133412,74343,135052,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Retweets,6490,10049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Replies,1767,3394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Quotes,844,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Likes,70049,118968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Tweets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,User,User Profile Url,User ID,Followers,Time Sent (Local),Local Date,Time Sent (GMT),Tweet ID,Tweet,Tweet URL,Interaction Type,Tweet Type,Root ID,Root URL
8,BlairBo88515781,https://twitter.com/BlairBo88515781,1.06534E+18,2,09:03:43 PM EST,12/9/18,2018-12-10T02:03:43Z,1.07195E+18,@narcicismo You are looking very sexy and beau...,https://twitter.com/statuses/1071948542854479873,engagement,reply,1.07177E+18,https://twitter.com/statuses/1071766108565315584
9,lovaticca2009,https://twitter.com/lovaticca2009,2767514403,1344,06:04:54 PM EST,12/9/18,2018-12-09T23:04:54Z,1.0719E+18,RT @UltimateKKW: I really hope KUWTK never ends,https://twitter.com/statuses/1071903545753042946,engagement,retweet,1.07178E+18,https://twitter.com/statuses/1071782454841634817


In [11]:
# calculate value for total mentions and add as column
mentions = nielsen.iloc[8:]
nielsen['total mentions'] = mentions['Unique Authors'].count()

nielsen.head()

17,nan,Unique Authors,Original Content,Unique Engagers,Total Engagement,Uniques,Interactions,nan,nan,nan,nan,nan,nan,nan,total mentions
0,Twitter,862,1640,73856,133412,74343,135052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16084
1,NaN,NaN,Retweets,6490,10049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16084
2,NaN,NaN,Replies,1767,3394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16084
3,NaN,NaN,Quotes,844,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16084
4,NaN,NaN,Likes,70049,118968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16084


In [12]:
# keep only relevant columns and drop all rows except first
nielsen = nielsen[['Unique Authors','Original Content','Unique Engagers','Interactions','total mentions']]
nielsen.drop(nielsen.index[1:], inplace=True)
nielsen.head()

17,Unique Authors,Original Content,Unique Engagers,Interactions,total mentions
0,862,1640,73856,135052,16084


In [13]:
# rename and reorder columns
nielsen.columns = ['unique users','total social convos','unique authors/engagers','interactions','total mentions']
nielsen['episode'], nielsen['name'] = episode_details['episode'], episode_details['name']
nielsen = nielsen[['episode','name','unique users','total social convos','unique authors/engagers','interactions','total mentions']]
nielsen.head()

,episode,name,unique users,total social convos,unique authors/engagers,interactions,total mentions
0,1516,Break Free,862,1640,73856,135052,16084


In [14]:
nielsen.to_csv('nielsen_weekly.csv')

### Facebook

Read in data export from Facebook platform, wrangle, and write to csv.

In [15]:
fb = pd.read_csv('fb_weekly_pull.csv')
print(fb.info())
fb.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 54 columns):
Post ID                                                                        11 non-null object
Permalink                                                                      11 non-null object
Post Message                                                                   11 non-null object
Type                                                                           11 non-null object
Countries                                                                      0 non-null float64
Languages                                                                      0 non-null float64
Posted                                                                         11 non-null object
Audience Targeting                                                             11 non-null object
Lifetime Post Total Reach                                                      12 non-null object
Lifetime Post o

,Post ID,Permalink,Post Message,Type,Countries,Languages,Posted,Audience Targeting,Lifetime Post Total Reach,Lifetime Post organic reach,...,Lifetime Matched Audience Targeting Consumptions by Type - video play,Lifetime Matched Audience Targeting Consumptions by Type - other clicks,Lifetime Matched Audience Targeting Consumptions by Type - link clicks,Lifetime Matched Audience Targeting Consumptions by Type - photo view,Lifetime Negative Feedback from Users by Type - hide_all_clicks,Lifetime Negative Feedback from Users by Type - hide_clicks,Lifetime Negative Feedback from Users by Type - report_spam_clicks,Lifetime Negative Feedback by Type - hide_all_clicks,Lifetime Negative Feedback by Type - hide_clicks,Lifetime Negative Feedback by Type - report_spam_clicks
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lifetime: The number of people who had your Pa...,Lifetime: The number of people who had your Pa...,...,Lifetime: The number of clicks anywhere in the...,NaN,NaN,NaN,Lifetime: The number of times people have give...,NaN,NaN,Lifetime: The number of people who have given ...,NaN,NaN
1,166720031891_10155869324316892,https://www.facebook.com/kuwtk/posts/101558693...,No one knows how to make Khloé 😂 quite like Th...,Video,NaN,NaN,12/11/18 9:17,,326548,326548,...,7487,25213.0,111.0,NaN,13,18.0,NaN,13,18.0,NaN
2,166720031891_10155861355461892,https://www.facebook.com/kuwtk/posts/101558613...,A bigger family means even bigger changes for ...,Video,NaN,NaN,12/10/18 9:00,,2291676,358586,...,49692,32064.0,NaN,NaN,15,28.0,1.0,15,28.0,1.0
3,166720031891_10155861301676892,https://www.facebook.com/kuwtk/posts/101558613...,"Dolls, who do you think was the MOST exciting ...",Video,NaN,NaN,12/9/18 17:00,,256686,256686,...,5307,14081.0,NaN,NaN,19,21.0,NaN,19,21.0,NaN
4,166720031891_10155861335196892,https://www.facebook.com/kuwtk/posts/101558613...,"‘Tis the holiday season, so tag that bestie yo...",Photo,NaN,NaN,12/9/18 10:30,,381003,381003,...,NaN,2842.0,NaN,10035.0,29,51.0,NaN,28,34.0,NaN


In [16]:
#drop unwanted rows and columns
fb.drop(fb.index[0], inplace=True)
fb = fb.filter(['Permalink','Post Message','Lifetime Post organic reach','Lifetime Post Organic Impressions',
            'Lifetime Engaged Users','Lifetime Organic Video Views.1','Lifetime Organic views to 95%.1',
                'Lifetime Talking About This (Post) by action type - like',
                'Lifetime Talking About This (Post) by action type - comment',
                'Lifetime Talking About This (Post) by action type - share'])
fb = fb.reset_index(drop=True)
fb.head()

,Permalink,Post Message,Lifetime Post organic reach,Lifetime Post Organic Impressions,Lifetime Engaged Users,Lifetime Organic Video Views.1,Lifetime Organic views to 95%.1,Lifetime Talking About This (Post) by action type - like,Lifetime Talking About This (Post) by action type - comment,Lifetime Talking About This (Post) by action type - share
0,https://www.facebook.com/kuwtk/posts/101558693...,No one knows how to make Khloé 😂 quite like Th...,326548,417931,24177,118720,12891,903.0,19.0,18
1,https://www.facebook.com/kuwtk/posts/101558613...,A bigger family means even bigger changes for ...,358586,459958,67721,133013,11914,2580.0,128.0,103
2,https://www.facebook.com/kuwtk/posts/101558613...,"Dolls, who do you think was the MOST exciting ...",256686,332642,15054,94689,39460,838.0,226.0,48
3,https://www.facebook.com/kuwtk/posts/101558613...,"‘Tis the holiday season, so tag that bestie yo...",381003,641661,10446,0,0,615.0,73.0,38
4,https://www.facebook.com/kuwtk/posts/101558590...,It’s not every day that you can change someone...,475160,602424,141440,148224,11735,3954.0,330.0,280


In [17]:
# rename columns
fb.columns = ['link','message','reach','impressions','engaged users','video views','video completes','likes','comments','shares']
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 10 columns):
link               11 non-null object
message            11 non-null object
reach              11 non-null object
impressions        11 non-null object
engaged users      11 non-null object
video views        11 non-null object
video completes    11 non-null object
likes              11 non-null float64
comments           11 non-null float64
shares             11 non-null object
dtypes: float64(2), object(8)
memory usage: 960.0+ bytes


In [18]:
#change data types

fb = fb.fillna(0)

cols = ['likes','comments','shares','reach','impressions','engaged users','video views','video completes']

for col in cols:
    fb[col] = fb[col].astype(int)
    
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 10 columns):
link               11 non-null object
message            11 non-null object
reach              11 non-null int64
impressions        11 non-null int64
engaged users      11 non-null int64
video views        11 non-null int64
video completes    11 non-null int64
likes              11 non-null int64
comments           11 non-null int64
shares             11 non-null int64
dtypes: int64(8), object(2)
memory usage: 960.0+ bytes


In [19]:
# add calculated columns
fb['PAID?'] = 'ORG'
fb['episode'] = 1516
fb['answers'] = np.nan
fb['vis eng'] = fb['likes'] + fb['comments'] + fb['shares']
fb['engagement rate'] = fb['engaged users']/fb['reach']
fb['view rate'] = fb['video views']/fb['impressions']
fb['completion rate'] = fb['video completes']/fb['impressions']
fb.head()

,link,message,reach,impressions,engaged users,video views,video completes,likes,comments,shares,PAID?,episode,answers,vis eng,engagement rate,view rate,completion rate
0,https://www.facebook.com/kuwtk/posts/101558693...,No one knows how to make Khloé 😂 quite like Th...,326548,417931,24177,118720,12891,903,19,18,ORG,1516,NaN,940,0.074038,0.284066,0.030845
1,https://www.facebook.com/kuwtk/posts/101558613...,A bigger family means even bigger changes for ...,358586,459958,67721,133013,11914,2580,128,103,ORG,1516,NaN,2811,0.188856,0.289185,0.025902
2,https://www.facebook.com/kuwtk/posts/101558613...,"Dolls, who do you think was the MOST exciting ...",256686,332642,15054,94689,39460,838,226,48,ORG,1516,NaN,1112,0.058648,0.284657,0.118626
3,https://www.facebook.com/kuwtk/posts/101558613...,"‘Tis the holiday season, so tag that bestie yo...",381003,641661,10446,0,0,615,73,38,ORG,1516,NaN,726,0.027417,0.000000,0.000000
4,https://www.facebook.com/kuwtk/posts/101558590...,It’s not every day that you can change someone...,475160,602424,141440,148224,11735,3954,330,280,ORG,1516,NaN,4564,0.297668,0.246046,0.019480


In [20]:
# reorder columns
fb = fb[['link','message','PAID?','episode','reach','impressions','shares','likes','comments','answers','vis eng','engaged users','engagement rate','video views','video completes','view rate','completion rate']]

fb.head()

,link,message,PAID?,episode,reach,impressions,shares,likes,comments,answers,vis eng,engaged users,engagement rate,video views,video completes,view rate,completion rate
0,https://www.facebook.com/kuwtk/posts/101558693...,No one knows how to make Khloé 😂 quite like Th...,ORG,1516,326548,417931,18,903,19,NaN,940,24177,0.074038,118720,12891,0.284066,0.030845
1,https://www.facebook.com/kuwtk/posts/101558613...,A bigger family means even bigger changes for ...,ORG,1516,358586,459958,103,2580,128,NaN,2811,67721,0.188856,133013,11914,0.289185,0.025902
2,https://www.facebook.com/kuwtk/posts/101558613...,"Dolls, who do you think was the MOST exciting ...",ORG,1516,256686,332642,48,838,226,NaN,1112,15054,0.058648,94689,39460,0.284657,0.118626
3,https://www.facebook.com/kuwtk/posts/101558613...,"‘Tis the holiday season, so tag that bestie yo...",ORG,1516,381003,641661,38,615,73,NaN,726,10446,0.027417,0,0,0.000000,0.000000
4,https://www.facebook.com/kuwtk/posts/101558590...,It’s not every day that you can change someone...,ORG,1516,475160,602424,280,3954,330,NaN,4564,141440,0.297668,148224,11735,0.246046,0.019480


In [21]:
fb.to_csv('fb_weekly.csv')

### IG Organic
Read in data export from Tracer, wrangle, and write to csv.

In [22]:
ig = pd.read_csv('tracer_ig_org_weekly_pull.csv')
print(ig.info())
ig.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 10 columns):
Page Name              10 non-null object
Post Created Date      9 non-null object
Post Permalink         9 non-null object
Post Caption           9 non-null object
Impressions            9 non-null float64
Reach                  9 non-null float64
Engagement             9 non-null float64
Post Like Count        9 non-null float64
Post Comments Count    9 non-null float64
Video Views            9 non-null float64
dtypes: float64(6), object(4)
memory usage: 960.0+ bytes
None


,Page Name,Post Created Date,Post Permalink,Post Caption,Impressions,Reach,Engagement,Post Like Count,Post Comments Count,Video Views
0,Kardashians on E!,12/6/18,https://www.instagram.com/p/BrDr0aNBC_J/,One day Kim heard Alice Johnson's story. The r...,450952.0,381131.0,14555.0,13126.0,72.0,223390.0
1,Kardashians on E!,12/7/18,https://www.instagram.com/p/BrGNsahn-RO/,Patience has a limit. A mom’s wrath does not 💅...,682713.0,575937.0,41010.0,39011.0,171.0,0.0
2,Kardashians on E!,12/7/18,https://www.instagram.com/p/BrGVDSGHq5f/,How much do you love these lewks? Copy one of ...,730785.0,622069.0,42233.0,40709.0,87.0,0.0
3,Kardashians on E!,12/8/18,https://www.instagram.com/p/BrIjpDZBCVQ/,Don't miss the glamorous @KimKardashian on @Bu...,300793.0,258989.0,14118.0,13904.0,51.0,0.0
4,Kardashians on E!,12/9/18,https://www.instagram.com/p/BrLIj8Ph_fY/,Stop scrolling ✋ Now watch the moment @KimKard...,477402.0,426115.0,16402.0,14862.0,85.0,221557.0


In [23]:
#drop unwanted columns
ig.drop(columns = ['Page Name','Post Created Date'], axis=1, inplace=True)
ig.head()

,Post Permalink,Post Caption,Impressions,Reach,Engagement,Post Like Count,Post Comments Count,Video Views
0,https://www.instagram.com/p/BrDr0aNBC_J/,One day Kim heard Alice Johnson's story. The r...,450952.0,381131.0,14555.0,13126.0,72.0,223390.0
1,https://www.instagram.com/p/BrGNsahn-RO/,Patience has a limit. A mom’s wrath does not 💅...,682713.0,575937.0,41010.0,39011.0,171.0,0.0
2,https://www.instagram.com/p/BrGVDSGHq5f/,How much do you love these lewks? Copy one of ...,730785.0,622069.0,42233.0,40709.0,87.0,0.0
3,https://www.instagram.com/p/BrIjpDZBCVQ/,Don't miss the glamorous @KimKardashian on @Bu...,300793.0,258989.0,14118.0,13904.0,51.0,0.0
4,https://www.instagram.com/p/BrLIj8Ph_fY/,Stop scrolling ✋ Now watch the moment @KimKard...,477402.0,426115.0,16402.0,14862.0,85.0,221557.0


In [24]:
#rename columns
ig.columns = ['link','caption','impressions','reach','engagement','likes','comments','video views']

#add columns
ig['episode'] = 1516
ig['vis eng'] = ig['likes'] + ig['comments']
ig['eng rate'] = ig['engagement']/ig['impressions']
ig['view rate'] = ig['video views']/ig['impressions']

#reorder columns
ig = ig[['link','caption','episode','impressions','reach','engagement','likes','comments','video views','vis eng','eng rate','view rate']]

print(ig.info())
ig.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 12 columns):
link           9 non-null object
caption        9 non-null object
episode        11 non-null int64
impressions    9 non-null float64
reach          9 non-null float64
engagement     9 non-null float64
likes          9 non-null float64
comments       9 non-null float64
video views    9 non-null float64
vis eng        9 non-null float64
eng rate       9 non-null float64
view rate      9 non-null float64
dtypes: float64(9), int64(1), object(2)
memory usage: 1.1+ KB
None


,link,caption,episode,impressions,reach,engagement,likes,comments,video views,vis eng,eng rate,view rate
0,https://www.instagram.com/p/BrDr0aNBC_J/,One day Kim heard Alice Johnson's story. The r...,1516,450952.0,381131.0,14555.0,13126.0,72.0,223390.0,13198.0,0.032276,0.495374
1,https://www.instagram.com/p/BrGNsahn-RO/,Patience has a limit. A mom’s wrath does not 💅...,1516,682713.0,575937.0,41010.0,39011.0,171.0,0.0,39182.0,0.060069,0.000000
2,https://www.instagram.com/p/BrGVDSGHq5f/,How much do you love these lewks? Copy one of ...,1516,730785.0,622069.0,42233.0,40709.0,87.0,0.0,40796.0,0.057791,0.000000
3,https://www.instagram.com/p/BrIjpDZBCVQ/,Don't miss the glamorous @KimKardashian on @Bu...,1516,300793.0,258989.0,14118.0,13904.0,51.0,0.0,13955.0,0.046936,0.000000
4,https://www.instagram.com/p/BrLIj8Ph_fY/,Stop scrolling ✋ Now watch the moment @KimKard...,1516,477402.0,426115.0,16402.0,14862.0,85.0,221557.0,14947.0,0.034357,0.464089


In [25]:
#write to csv
ig.to_csv('tracer_ig_org_weekly.csv')